In [159]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
df = pd.read_csv('5YearPreds.csv')
df.head()

,Unnamed: 0,Player,PTS2,BLK2,STL2,AST2,TRB2,Value,Year
0,188,Aaron Gordon,16.0,0.7,0.7,3.7,7.4,34.63,2019
1,228,Aaron Holiday,5.9,0.3,0.4,1.7,1.3,12.11,2019
2,370,Abdel Nader,4.0,0.2,0.3,0.3,1.9,8.23,2019
3,235,Al Horford,13.6,1.3,0.9,4.2,6.7,34.54,2019
4,12,Al-Farouq Aminu,9.4,0.4,0.8,1.3,7.5,23.95,2019


In [160]:
df = df.drop(['Unnamed: 0'], axis=1)

In [161]:
df = df.rename(index=str, columns={"PTS2": "Points", "BLK2": "Blocks", "STL2": "Steals", "AST2": "Assists", "TRB2": "Rebounds"})
df.head()

,Player,Points,Blocks,Steals,Assists,Rebounds,Value,Year
0,Aaron Gordon,16.0,0.7,0.7,3.7,7.4,34.63,2019
1,Aaron Holiday,5.9,0.3,0.4,1.7,1.3,12.11,2019
2,Abdel Nader,4.0,0.2,0.3,0.3,1.9,8.23,2019
3,Al Horford,13.6,1.3,0.9,4.2,6.7,34.54,2019
4,Al-Farouq Aminu,9.4,0.4,0.8,1.3,7.5,23.95,2019


In [162]:
import re
df['Player'] = df['Player'].map(lambda x: re.sub('\.', '', x))
df = df.set_index('Player')
df = df.round(2)
df.head()

,Points,Blocks,Steals,Assists,Rebounds,Value,Year
Player,,,,,,,
Aaron Gordon,16.0,0.7,0.7,3.7,7.4,34.63,2019
Aaron Holiday,5.9,0.3,0.4,1.7,1.3,12.11,2019
Abdel Nader,4.0,0.2,0.3,0.3,1.9,8.23,2019
Al Horford,13.6,1.3,0.9,4.2,6.7,34.54,2019
Al-Farouq Aminu,9.4,0.4,0.8,1.3,7.5,23.95,2019


In [166]:
playerjs = (df.groupby(['Player'], as_index=True)
             .apply(lambda x: x[['Year','Points','Assists','Rebounds','Blocks','Steals','Value']].to_dict('r'))
             .reset_index()
             .rename(columns={0:'Stats'})
             .set_index('Player')
             .to_json(orient='index'))

In [171]:
import json
#print(json.dumps(json.loads(playerjs), indent=2, sort_keys=False))

In [168]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [169]:
db = client.NBAPredictions

In [170]:
playerStats = db['stats']

with open('playerAPI.json') as f:
    file_data = json.load(f)

# use collection_currency.insert(file_data) if pymongo version < 3.0
playerStats.insert_one(file_data)  
client.close()